# Power-system State-Estimation using Gurobi 

This notebook will take you through the steps required to perform custom Weighted Least Squares State-Estimation for a few pandapower networks. 

Methodology: 

1. [Load a 4-bus example synthetic network.](#section-title1) 


2. [Initialize the variables, parameters, constraints and objective function.](#section-title2) 


3. [Solve the model using IPOPT and check solver status.](#section-title3) 

In [1]:
import pyomo.environ as pyo
import pandapower as pp 
import numpy as np 
import pandas as pd 
import math 
from utils import get_branch_parameters, get_edge_index_from_ppnet, get_neighbor_dict


<a id="section-title1"></a>
## 1. Load a 4-bus example synthetic network.

<img src="4_bus_network/4_bus_network.001.jpeg" width="500">

In [2]:
net = pp.create_empty_network()

bus1 = pp.create_bus(net, vn_kv=20.0, name="Bus 1")
bus2 = pp.create_bus(net, vn_kv=20.0, name="Bus 2")
bus3 = pp.create_bus(net, vn_kv=0.4, name="Bus 3")  # connected via transformer
bus4 = pp.create_bus(net, vn_kv=20.0, name="Bus 4")

nodes = net.bus.index.values
num_buses = len(net.bus.index)

# standard 20 kV line type 
std_line_type = "NAYY 4x50 SE"

# Create standard lines
pp.create_line(net, from_bus=bus1, to_bus=bus2, length_km=1.0,
               std_type=std_line_type, name="Line 1-2")

pp.create_line(net, from_bus=bus1, to_bus=bus4, length_km=1.0,
               std_type=std_line_type, name="Line 1-4")

pp.create_line(net, from_bus=bus2, to_bus=bus4, length_km=1.0,
               std_type=std_line_type, name="Line 2-4")

pp.create_transformer(net, bus2, bus3, std_type="0.25 MVA 20/0.4 kV", name="Trafo 2-3")
net.trafo.shift_degree = 0.0 # bug in pandapwer 

pp.create_ext_grid(net, bus=bus2, vm_pu=1.02, name="Grid Connection")

pp.create_load(net, bus3, p_mw=0.2, q_mvar=0.05, name="Load Bus 3")
pp.create_load(net, bus4, p_mw=0.3, q_mvar=0.1, name="Load Bus 4")

pp.runpp(net)

edge_index = get_edge_index_from_ppnet(net=net)

<a id="section-title2"></a>
## 2. Initialize the variables, parameters, constraints and objective function.


In [3]:
# initialize the model 
model = pyo.ConcreteModel("WLS_State_Estimation")

#################### Parameters ####################
# parameters 
model.buses = pyo.Set(initialize=range(num_buses))

# measurement vector as 5% tolerance to power flow results 
V_meas = np.random.normal(net.res_bus.vm_pu.values, 0.05/3)
P_meas = np.random.normal(net.res_bus.p_mw.values, 0.05/3)

# create parameters for measurements
model.V_meas = pyo.Param(model.buses, initialize={i: V_meas[i] for i in range(num_buses)}, mutable=True)
model.P_meas = pyo.Param(model.buses, initialize={i: P_meas[i] for i in range(num_buses)}, mutable=True)

# get line and transformer parameters
# param_dict = {(i, j): [G_ij, B_ij, g_s_ij, b_s_ij, g_sh_ij, b_sh_ij]}
param_dict = get_branch_parameters(net=net) 


# get neighbor_dictionary 
# neighbor_dict = {src: [neighbors]}
neighbor_dict = get_neighbor_dict(edge_index)

#################### Decision Variables ####################
# decision variables
model.vm_pu = pyo.Var(model.buses, initialize=1.0)  # Voltage magnitude
model.va_rad = pyo.Var(model.buses, initialize=0.0)  # Voltage angle

# fix slack bus angle to zero
model.va_rad[0].fix(0.0)

# Add realistic bounds for voltage magnitudes (typically 0.9-1.1 p.u.)
for i in model.buses:
    model.vm_pu[i].setlb(0.9)
    model.vm_pu[i].setub(1.1)
    if i != 0:  # Skip the slack bus which is already fixed
        model.va_rad[i].setlb(-math.pi)
        model.va_rad[i].setub(math.pi)

#################### Constraints ####################  
# power injection equation
def power_mismatch_rule(model, i):
    P_calc = 0.0
    for j in neighbor_dict[i]:
        j = int(j)
        G_ij, B_ij = param_dict[(i,j)][0], param_dict[(i,j)][1]
        P_calc += model.vm_pu[i] * model.vm_pu[j] * (
            G_ij * pyo.cos(model.va_rad[i] - model.va_rad[j]) + 
            B_ij * pyo.sin(model.va_rad[i] - model.va_rad[j])
        )
    return model.P_meas[i] + P_calc

model.P_mismatch = pyo.Expression(model.buses, rule=power_mismatch_rule)

# voltage equation 
def voltage_mismatch_rule(model, i):
    return model.vm_pu[i] - model.V_meas[i]

model.V_mismatch = pyo.Expression(model.buses, rule=voltage_mismatch_rule)

#################### Objective Function ####################
# Define objective function (currently not WLS but Simple Sum for PoC)
def objective_rule(model):
    power_sum = sum(model.P_mismatch[i]**2 for i in model.buses)
    voltage_sum = sum(model.V_mismatch[i]**2 for i in model.buses)
    return power_sum + voltage_sum

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)


<a id="section-title3"></a>
## 3. Solve the model using IPOPT and check solver status.

In [4]:
#################### IPOPT Solver Pyomo ####################
# solve the model using IPOPT, GurobiPy does not support nonlinear objective function.
solver = pyo.SolverFactory('ipopt')
solver_results = solver.solve(model, tee=True)

#################### Solver Status ####################
# From documentation, for better troubleshooting and to check solver status
if solver_results.solver.status == pyo.SolverStatus.ok and \
   solver_results.solver.termination_condition == pyo.TerminationCondition.optimal:
    print("Optimal solution found")
    
    # Extract results
    estimated_voltages = {i: pyo.value(model.vm_pu[i]) for i in model.buses}
    estimated_angles = {i: pyo.value(model.va_rad[i]) for i in model.buses}
    
    # Print results
    print("Estimated Voltages (p.u.):")
    for i, v in estimated_voltages.items():
        print(f"Bus {i}: {v:.4f}")
        
    print("\nEstimated Angles (rad):")
    for i, a in estimated_angles.items():
        print(f"Bus {i}: {a:.4f}")
else:
    print("Solver did not converge to an optimal solution")

Ipopt 3.14.17: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.17, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       28

Total number of variables............................:        7
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        7
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number